In [1]:
using LinearAlgebra
using PyPlot
using RandomMatrix
using ITensors
using TensorOperations
using LsqFit

# Numerical many-body methods - Problem Set 3 
## (DMRG)
Noam Ophir

# $J_1-J_2$ spin-1/2 chain
*In this problem we will revisit the spin-1/2 antiferromagnetic $J_1-J_2$ chain discussed in Problem Set 1, given by the Hamiltonian*
$$
H
=
J_1\sum_i\vec{S}_i\cdot\vec{S}_{i+1}+J_2\sum_i\vec{S}_i\cdot\vec{S}_{i+2}
$$
*Recall that the model exhibits a phase transition between a gapless and a dimerized phase at $g_c\simeq 0.241$, where $g$ is the dimensionless ratio $J_2/J_1$.*

*We will now solve the problem using DMRG and employing the ITensor library. In sections 2,3 below use values of $g\leq1/2$. (For $g > 1/2$ the system is still dimerized but exhibitis incommensurate spin-spin correlations, reminiscent of the classical ordering, which can give rise to large finite-size effects.)*

## 1-1
*Generalize the construction of the Heisenberg Hamiltonian discussed in class to allow for a finite $J_2$ term.*

In [ ]:
function J1J2Hamiltonian(sites, J2::Float)
    # Initialize an operator sum representing the Hamiltonian using ITensors
    ampo = OpSum()

    # Construct the Hamiltonian operator for the 1D quantum system
    for j = 1:N-1
        # Heisenberg interaction term (Sz-Sz)
        ampo += "Sz", j, "Sz", j+1
        # Spin-flip terms (S+, S- and S-, S+) with coupling strength 1/2
        ampo += 1/2, "S+", j, "S-", j+1
        ampo += 1/2, "S-", j, "S+", j+1
    end
    for j = 1:N-2
        # Heisenberg interaction term (Sz-Sz)
        ampo += J2, "Sz", j, "Sz", j+2
        # Spin-flip terms (S+, S- and S-, S+) with coupling strength 1/2
        ampo += J2/2, "S+", j, "S-", j+2
        ampo += J2/2, "S-", j, "S+", j+2
    end

    # Convert the operator sum to a Matrix Product Operator (MPO)
    H = MPO(ampo, sites)
    return H
end

# Number of sites in the 1D quantum system
N = 100

# Generate an index set representing quantum sites with spin S=1/2, conserving quantum numbers
sites = siteinds("S=1/2", N; conserve_qns=true)

# Initialize an operator sum representing the Hamiltonian using ITensors
ampo = OpSum()


# Initialize the initial state to an antiferromagnetic (AFM) state with total Sz=0
+psi0 = productMPS(sites, state)

# Display the total flux of the initial state
@show flux(psi0)

# Set up parameters for the DMRG (Density Matrix Renormalization Group) algorithm
sweeps = Sweeps(10)
setmaxdim!(sweeps, 20, 50, 100)
setcutoff!(sweeps, 1E-12)

# Perform the DMRG simulation to obtain the ground state energy and MPS representation
energy, psi = dmrg(H, psi0, sweeps)


In [12]:
# Number of sites in the 1D quantum system
N = 100

# Generate an index set representing quantum sites with spin S=1/2, conserving quantum numbers
sites = siteinds("S=1/2", N; conserve_qns=true)

# Initialize an operator sum representing the Hamiltonian using ITensors
ampo = OpSum()

# Construct the Hamiltonian operator for the 1D quantum system
for j = 1:N-1
    # Heisenberg interaction term (Sz-Sz)
    ampo += "Sz", j, "Sz", j+1
    # Spin-flip terms (S+, S- and S-, S+) with coupling strength 1/2
    ampo += 1/2, "S+", j, "S-", j+1
    ampo += 1/2, "S-", j, "S+", j+1
end

# Convert the operator sum to a Matrix Product Operator (MPO)
H = MPO(ampo, sites)

state0 = [isodd(n) ? "Up" : "Dn" for n=1:N]
# Initialize the initial state to an antiferromagnetic (AFM) state with total Sz=0
psi0 = randomMPS(sites, state0,20)

# Display the total flux of the initial state
@show flux(psi0)

# Set up parameters for the DMRG (Density Matrix Renormalization Group) algorithm
sweeps = Sweeps(10)
setmaxdim!(sweeps, 20, 50, 100)
setcutoff!(sweeps, 1E-12)

# Perform the DMRG simulation to obtain the ground state energy and MPS representation
energy, psi = dmrg(H, psi0; sweeps)


LoadError: MethodError: no method matching svd(::NDTensors.BlockSparseTensor{Float64, 2, Tuple{Index{Vector{Pair{QN, Int64}}}, Index{Vector{Pair{QN, Int64}}}}, NDTensors.BlockSparse{Float64, Vector{Float64}, 2}}; maxdim=20, utags="Link,l=0")
[0mClosest candidates are:
[0m  svd(::NDTensors.BlockSparseTensor{ElT, 2, IndsT, var"#s24"} where {var"#s24"<:NDTensors.BlockSparse, IndsT}; min_blockdim, mindim, maxdim, cutoff, alg, use_absolute_cutoff, use_relative_cutoff) where ElT at /Users/noamophir/.julia/packages/NDTensors/RDn3T/src/blocksparse/linearalgebra.jl:38[91m got unsupported keyword argument "utags"[39m
[0m  svd([91m::StridedMatrix{T}[39m, [91m::StridedMatrix{T}[39m) where T<:Union{Float32, Float64, ComplexF32, ComplexF64} at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/svd.jl:370[91m got unsupported keyword arguments "maxdim", "utags"[39m
[0m  svd([91m::StridedVecOrMat{T}[39m; full, alg) where T at /Users/julia/buil

MPS
[1] ((dim=2|id=629|"S=1/2,Site,n=1") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=1|id=61|"Link,l=1") <In>
 1: QN("Sz",1) => 1)
[2] ((dim=1|id=61|"Link,l=1") <Out>
 1: QN("Sz",1) => 1, (dim=2|id=198|"S=1/2,Site,n=2") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=1|id=214|"Link,l=2") <In>
 1: QN("Sz",0) => 1)
[3] ((dim=1|id=214|"Link,l=2") <Out>
 1: QN("Sz",0) => 1, (dim=2|id=605|"S=1/2,Site,n=3") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=1|id=389|"Link,l=3") <In>
 1: QN("Sz",1) => 1)
[4] ((dim=1|id=389|"Link,l=3") <Out>
 1: QN("Sz",1) => 1, (dim=2|id=937|"S=1/2,Site,n=4") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=1|id=161|"Link,l=4") <In>
 1: QN("Sz",0) => 1)
[5] ((dim=1|id=161|"Link,l=4") <Out>
 1: QN("Sz",0) => 1, (dim=2|id=723|"S=1/2,Site,n=5") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1, (dim=1|id=186|"Link,l=5") <In>
 1: QN("Sz",1) => 1)
[6] ((dim=1|id=186|"Link,l=5") <Out>
 1: QN("Sz",1) => 1, (dim=2|id=929|"S=1/2,Site,n=6") <Out>
 1: Q

## 1-2
*Perform finite-size scaling of the triplet gap (i.e. the energy gap between the ground state in the $S^z = 0$ sector to the lowest state in the $S^z = 1$ sector) for different values of $g$ across the phase transition. To find the lowest energy state in each $S^z$ sector you will need to make sure that the initial state is in the same $S^z$ sector. You can now easily access system sizes of up to $N = 100$ sites (performing e.g. 10 sweeps with bond dimensions of order 100), but to clearly observe a finite gap for $g > g_c$ it is still advisable to consider values of g close enough to $g = 1/2$.*

## 1-3
*Calculate the spin-spin correlations (i.e. the expectation value of $\langle\vec{S}_{x_0} \cdot \vec{S}_{x_0+x} \rangle$ as function of $x$) for different values of g across the phase transition. Show that for $g\leq g_c$ the correlations decay as a power law (i.e.  $1/x^\alpha$), while for $g > g_c$ the decay is exponential (i.e. $\sim\exp(-x\xi)$) . Use system sizes of $N = 200$ sites and calculate the correlations with respect to a site in the middle of the system to avoid boundary effects. (Recall that we are now working with open boundary conditions as opposed to all the calculations we performed using Exact Diagonalization.) For $g\leq g_c$ calculate the exponent $\alpha$, and for $g > g_c$ obtain the correlation length $\xi$. Note: to obtain the spin-spin correlations you can use the built in ITensor function `correlation matrix(psi,"Sz","Sz")` that calculates the full correlation matrix $\langle \psi|S^z_i S^z_j|\psi \rangle$ for every pair of sites $(i,j)$ in the system.*

# Spin-1 chain - Haldane gap
*In this problem we will consider the spin-1 bilinear-biquadtatic model*
$$H=\sum_i J_1 \vec{S}_i\cdot\vec{S}_{i+1} + J_2 \left( \vec{S}_i\cdot\vec{S}_{i+1} \right)^2 $$
*As discussed in the tutorial and in Problem Set 2, for $J_2/J_1 = 1/3$ the model is exactly solvable,and the ground state can be written explicitly as an MPS of bind dimension $M = 2$. At this point you observed that the ground state is four-fold degenerate for open boundary conditions and that the edges feature fractionalized spin-1/2 states.*

*In this problem you will show that when $J_2\rightarrow0$ the system remains in the same phase, i.e. it is gapped, and hosts fractionalized edge mods.*

## 2-1
*Consider different values of $J_2$ ranging between $1/3$ and $0$ and show that the system remains gapped within this range. To this end, perform finite-size scaling of the bulk gap. Note that to probe the bulk gap in this case you will need to look at the $S^z = 2$ sector, as the lowest energy state in the $S^z = 1$ sector simply accumulates the extra spin at the edges of the system (you'll check this in the next section).*

## 2-2
*For the same values of $J_2$ as above, calculate the gap between the $S^z = 1$ and $S^z = 0$ sectors.Show that the gap decays exponentially with system size (i.e. the states are degenerate in the thermodynamic limit). Calculate the local expectation value of $S^z$ along the chain in the lowest energy state with $S^z = 1$ and show that the total spin accumulated near each edge is $\pm 1/2$.*